In [1]:
import numpy as np
import pandas as pd

---
### Setting up basic matrices:
The Pauli matrices are the four 2x2 matrices, I, X, Y, Z.
For this example, we have a 4x4 Hamiltonian that is decomposed into IZ, ZI, XX, YY, and ZZ terms (this notation is not matrix multiplication--it is a Kronecker product, which takes two 2x2 matrices and gives a 4x4 matrix).

In [2]:
I = np.eye(2)
X = np.array([[0., 1.],
              [1., 0.]])
Y = np.array([[0., -1j],
              [1j, 0.]])
Z = np.array([[1., 0.],
              [0., -1.]])

In [3]:
IZ = np.kron(I, Z)
print(IZ)

[[ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0.  1.  0.]
 [ 0. -0.  0. -1.]]


In [4]:
ZI = np.kron(Z, I)
print(ZI)

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1. -0.]
 [ 0.  0. -0. -1.]]


In [5]:
XX = np.kron(X, X)
print(XX)

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [6]:
YY = np.kron(Y, Y)
print(YY)

[[ 0.+0.j  0.-0.j  0.-0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  1.-0.j  0.-0.j]
 [ 0.+0.j  1.-0.j  0.+0.j  0.-0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]]


In [7]:
ZZ = np.kron(Z, Z)
print(ZZ)

[[ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0. -1. -0.]
 [ 0. -0. -0.  1.]]


----
### Hamiltonian
We consider the Hamiltonian:
$H = IZ + ZI - XX - YY + ZZ$

In [8]:
H = IZ + ZI - XX - YY + ZZ
print(H)

[[ 3.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]]


---
### Measuring an observable
An observable is a matrix that we can measure against a state $\vec{\psi}$. The Hamiltonian is an observable. So are the Pauli terms that it is written as a sum of.

To measure some observable matrix $M$ with respect to a state $\vec{\psi}$, we just compute $\vec{\psi}^{\dagger} M \vec{\psi}$. The $\dagger$ means conjugate transpose = transpose the matrix and then take complex conjugates. In this specific example, all the numbers are real, so it happens to be the same as an ordinary matrix transpose.

Here is a helper method that does everything for us:

In [9]:
def expected_value(M, state):
    return np.asmatrix(state).getH() @ M @ state  # .getH() does conjugate tranpose

---
### Let's try for $\vec{\psi} = [0, 1, 0, 0]^T$

In [10]:
state = np.array([[0.], [1.], [0.], [0.]])

#### Method 1: measure $H$ directly
A quantum computer can't actually measure $H$ directly, but let's just check what the answer should be.

In [11]:
print(expected_value(H, state))

[[-1.+0.j]]


#### Method 2: linearity of expectation
A quantum computer can compute the Pauli observables though. So let's use linearity of expectation instead:

In [12]:
IZ_expectation = expected_value(IZ, state)
ZI_expectation = expected_value(ZI, state)
XX_expectation = expected_value(XX, state)
YY_expectation = expected_value(YY, state)
ZZ_expectation = expected_value(ZZ, state)
print(IZ_expectation, ZI_expectation, XX_expectation, YY_expectation, ZZ_expectation)
print(IZ_expectation + ZI_expectation - XX_expectation - YY_expectation + ZZ_expectation)

[[-1.]] [[1.]] [[0.]] [[0.+0.j]] [[-1.]]
[[-1.+0.j]]


Yup, we got -1 both ways!

---
### Variances / Covariances

In [13]:
def variance(M, state):
    # variances is <M^2> - <M>^2
    return expected_value(M @ M, state) - expected_value(M, state) ** 2

def covariance(M_1, M_2, state):
    return expected_value(M_1 @ M_2, state) - expected_value(M_1, state) * expected_value(M_2, state)

The cells below show that:
- Var(H) = 4
- Var(IZ) + Var(ZI) + Var(XX) + Var(YY) + Var(ZZ) = 2 -- not equal because we need to account for covariances
- the summed covariances do indeed equal 4. Note that the XX and ZZ

In [14]:
variance(H, state)

matrix([[4.+0.j]])

In [15]:
variance(IZ, state)

matrix([[0.]])

In [16]:
variance(ZI, state)

matrix([[0.]])

In [17]:
variance(XX, state)

matrix([[1.]])

In [18]:
variance(YY, state)

matrix([[1.+0.j]])

In [19]:
variance(ZZ, state)

matrix([[0.]])

In [20]:
terms, coefficients = [IZ, ZI, XX, YY, ZZ], [1., 1., -1., -1., 1.]
total = np.array([[0.+0.j]])
for i in range(5):
    for j in range(5):
        cov = covariance(terms[i], terms[j], state)
        if cov != 0:
            print(cov, i, j)
            total += (coefficients[i] * coefficients[j] * covariance(terms[i], terms[j], state))
print('Variance from summed covariances is %s' % total)

[[1.]] 2 2
[[1.+0.j]] 2 3
[[1.+0.j]] 3 2
[[1.+0.j]] 3 3
Variance from summed covariances is [[4.+0.j]]


----
### Different State, $\vec{\psi} = [0.43 - 0.16i, -0.49, 0.44i, -0.39 + 0.46i]$

In [21]:
state = np.array([[0.43 - 0.16j], [-0.49], [0.43j], [-0.39 + 0.46j]])

In [22]:
variance(H, state)

matrix([[4.35848816-2.22044605e-16j]])

In [23]:
variance(IZ, state), variance(ZI, state), variance(-XX, state), variance(-YY, state), variance(ZZ, state)

(matrix([[0.95576944+0.j]]),
 matrix([[0.989596+0.j]]),
 matrix([[0.76629724+6.6974204e-18j]]),
 matrix([[0.76629724+0.j]]),
 matrix([[0.97693936+0.j]]))

In [24]:
print(covariance(IZ, ZI, state))
print(covariance(IZ, ZZ, state))
print(covariance(ZI, ZZ, state))
print(covariance(-XX, -YY, state))
print(covariance(-XX, ZZ, state))
print(covariance(-YY, ZZ, state))

[[0.1287768+0.j]]
[[-0.06690672+0.j]]
[[-0.1937784+0.j]]
[[0.08370276-3.3487102e-18j]]
[[0.41059608+1.03528297e-18j]]
[[-0.41059608+6.9388939e-18j]]


In [25]:
# Setting 1 (measure all terms separately) would require a number of state preparations equal to:
5 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936)

22.2744964

In [26]:
# Setting 2 (measure {XX}, {YY, ZZ}, {IZ, ZI}) would require a number of state preparations equal to:
3 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936 + 2*0.08370276 + 2*0.1287768)

14.6395752

In [27]:
# Setting 3 (measure {XX, YY, ZZ}, {IZ, ZI}) would require a number of state preparations equal to:
2 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936 + 2*0.08370276 + 2*0.41059608 + 2*-0.41059608 + 2*0.1287768)

9.7597168

### Programatically finding optimal groupings

In [28]:
def variance_grouping(grouping, state):
    """
    Computes the variance of a grouping, via the formula
             Var(A+B) = Var(A) + Var(B) + 2*Cov(A,B)
    Inputs:
       Grouping (as a list, e.g. [-XX, -YY, ZZ])
       State (as a np.array, e.g. np.array([[0.], [1.], [0.], [0.]]))
    """
    running_variance = sum([variance(term, state) for term in grouping])
    grouping_size = len(grouping)
    for i in range(grouping_size):
        for j in range(i):
            running_variance += 2*covariance(grouping[i], grouping[j], state)
    return(np.asscalar(running_variance))

In [29]:
# # Credit to: https://stackoverflow.com/questions/19368375/set-partitions-in-python
def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller


In [30]:
def grouping_is_pairwise_commuting(grouping_to_test):
    """
    Computes whether all pairs within a grouping commute (in a general sense).
    Inputs:
        grouping_to_test: a single list of *strings*, e.g. ["-XX", "-YY", "ZZ", "IZ", "ZI"]
    Returns:
        True if all pairs commute; False if not all pairs commute
    """
    evaled_grouping_to_test = [eval(term) for term in grouping_to_test]
    running_sum = 0
    for i in range(len(evaled_grouping_to_test)):
        for j in range(i+1, len(evaled_grouping_to_test)):
            # RHS evaluates to 1 if non-commuting, 0 if commuting
            running_sum += not np.array_equal(evaled_grouping_to_test[i] @ evaled_grouping_to_test[j],
                               evaled_grouping_to_test[j] @ evaled_grouping_to_test[i])
    # sum will be 0 iff all pairs commute
    return(running_sum == 0)

def is_valid_partition(partition):
    """
    Computes whether all groupings within a partition are pairwise commuting.
    Inputs:
        partition: a list of lists of strings, e.g. [['-XX', '-YY'], ['ZZ', 'IZ', 'ZI']]
    Outputs:
        1 if all groupings within the partition are pairwise commuting. Else 0.
    """
    return(sum([grouping_is_pairwise_commuting(grouping) for grouping in partition]) == len(partition))

In [33]:
def calculate_variance_valid_partitions(term_set, state):
    """
    Computes the variance of every valid partition of a set of terms.
    A valid partition is one in which all groupings are pairwise commuting.
    Inputs:
        term_set: list of *strings*, e.g. ["-XX", "-YY", "ZZ", "IZ", "ZI"]
        state: as a np.array, e.g. np.array([[0.], [1.], [0.], [0.]]))
    """
    evaled_term_set = [eval(term) for term in term_set]
    term_list = []
    variance_list = []
    for pp in partition(list(range(len(term_set)))):
         term_list.append([list(np.array(term_set)[grouping]) for grouping in pp])
         variance_list.append(sum([variance_grouping(np.array(evaled_term_set)[indices], state)*len(pp) for indices in pp]))
    partition_list = pd.DataFrame({'partition': term_list, 'variance': variance_list}).sort_values(by=["variance"])
    # Eliminate invalid partitions that contain groupings that are non-pairwise-computing.
    return(partition_list[partition_list['partition'].apply(lambda partition: is_valid_partition(partition))].reset_index(drop=True))

# Aspuru-Guzik example
test_term_set = ["-XX", "-YY", "ZZ", "IZ", "ZI"]
state = np.array([[0], [1], [0], [0]])

partition_list = calculate_variance_valid_partitions(test_term_set, state)
partition_list


,partition,variance
0,"[[-XX], [-YY, ZZ], [IZ, ZI]]",(6+0j)
1,"[[-YY], [-XX, ZZ], [IZ, ZI]]",(6+0j)
2,"[[-XX], [-YY], [ZZ, IZ, ZI]]",(6+0j)
3,"[[-YY], [-XX, ZZ], [IZ], [ZI]]",(8+0j)
4,"[[-XX], [-YY, ZZ], [IZ], [ZI]]",(8+0j)
5,"[[-XX], [-YY], [IZ], [ZZ, ZI]]",(8+0j)
6,"[[-XX], [-YY], [ZZ], [IZ, ZI]]",(8+0j)
7,"[[-XX], [-YY], [ZZ, IZ], [ZI]]",(8+0j)
8,"[[-XX, -YY], [ZZ, IZ, ZI]]",(8+0j)
9,"[[-XX, -YY, ZZ], [IZ, ZI]]",(8+0j)
